#1. Contexto do Projeto (Introdução)
Este projeto de dashboard foi desenvolvido com o objetivo de ajudar a equipe da CHIVUNK a tomar decisões estratégicas baseadas em dados sobre o desempenho dos alunos, visando aumentar a taxa de aprovação nos concursos militares.

#2. Problemas Identificados
Dificuldade em visualizar quais métodos de estudo dão mais resultado.

Falta de dados claros sobre engajamento e desempenho dos alunos por estado.

Dificuldade em correlacionar tempo de estudo com pontuação.

Falta de uma ferramenta simples que permita filtrar e cruzar variáveis.



#3. Solução Proposta
# Para resolver esses pontos, desenvolvi um dashboard interativo com métricas e gráficos que mostram padrões de desempenho, uso de materiais e engajamento por estado. O dashboard permite identificar oportunidades de melhoria na metodologia e foco dos alunos.

# 4. Objetivos
Ajudar a entender quais modalidades de estudo têm maior taxa de aprovação.
Verificar se o tempo de estudo está correlacionado com maior pontuação nos simulados.
Medir o nível médio de engajamento e tempo de estudo.
Permitir que a equipe faça análises por estado ou por material usado.


#5. Resultado
Com esse dashboard, a CHIVUNK consegue rapidamente identificar quais estratégias estão funcionando melhor, focar seus esforços de mentoria, e tomar decisões baseadas em dados com muito mais agilidade.

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
from google.colab import files
uploaded = files.upload()

Saving dados_chivunk.csv to dados_chivunk (1).csv


In [14]:

# Carregar os dados
df = pd.read_csv('dados_chivunk.csv')

# Inicializar o app Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout do dashboard
app.layout = dbc.Container([
    dbc.Row([
        html.H1("📊 Dashboard de Desempenho - Chivunk", className="text-center my-4"),
    ]),

    # KPIs Rápidos
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Taxa de Aprovação", className="text-center"),
                html.H3(f"{df[df['Resultado no Concurso'] == 'Aprovado'].shape[0] / df.shape[0] * 100:.1f}%",
                        className="text-center text-success")
            ], className="p-3 border rounded")
        ], md=4),

        dbc.Col([
            html.Div([
                html.H4("Média de Tempo de Estudo (h/dia)", className="text-center"),
                html.H3(f"{df['Tempo de Estudo (h/dia)'].mean():.1f}",
                        className="text-center text-primary")
            ], className="p-3 border rounded")
        ], md=4),

        dbc.Col([
            html.Div([
                html.H4("Engajamento Médio (1-5)", className="text-center"),
                html.H3(f"{df['Engajamento com a Tribo (1 a 5)'].mean():.1f}",
                        className="text-center text-warning")
            ], className="p-3 border rounded")
        ], md=4),
    ], className="mb-4"),

    # Gráficos Interativos
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='dropdown-estado',
                options=[{'label': estado, 'value': estado} for estado in df['Estado'].unique()],
                multi=True,
                placeholder="Filtrar por Estado..."
            ),
            dcc.Graph(id='grafico-aprovacao-modalidade')
        ], md=6),

        dbc.Col([
            dcc.Dropdown(
                id='dropdown-material',
                options=[{'label': material, 'value': material} for material in df['Material Utilizado'].unique()],
                placeholder="Filtrar por Material..."
            ),
            dcc.Graph(id='grafico-tempo-pontuacao')
        ], md=6),
    ]),

    # Tabela Detalhada
    dbc.Row([
        html.H4("Dados Detalhados", className="mt-4"),
        html.Div(id='tabela-dados', className="mb-4")
    ])
])

# Callbacks para interatividade
@app.callback(
    Output('grafico-aprovacao-modalidade', 'figure'),
    Input('dropdown-estado', 'value')
)
def update_modalidade_plot(estados_selecionados):
    filtered_df = df if not estados_selecionados else df[df['Estado'].isin(estados_selecionados)]
    fig = px.bar(
        filtered_df,
        x='Modalidade de Estudo',
        color='Resultado no Concurso',
        barmode='group',
        title='Aprovação por Modalidade de Estudo'
    )
    return fig

@app.callback(
    Output('grafico-tempo-pontuacao', 'figure'),
    Input('dropdown-material', 'value')
)
def update_scatter_plot(material):
    filtered_df = df if not material else df[df['Material Utilizado'] == material]
    fig = px.scatter(
        filtered_df,
        x='Tempo de Estudo (h/dia)',
        y='Pontuação em Simulados',
        color='Resultado no Concurso',
        title='Relação: Tempo de Estudo vs. Pontuação'
    )
    return fig

@app.callback(
    Output('tabela-dados', 'children'),
    [Input('dropdown-estado', 'value'),
     Input('dropdown-material', 'value')]
)
def update_table(estados, material):
    filtered_df = df
    if estados:
        filtered_df = filtered_df[filtered_df['Estado'].isin(estados)]
    if material:
        filtered_df = filtered_df[filtered_df['Material Utilizado'] == material]
    return dbc.Table.from_dataframe(filtered_df, striped=True, bordered=True)

# Executar o app
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>